# 1. News crawling

In [ ]:
import numpy as np
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time


# 크롤링 할 크롬 창 오픈
URL = 'https://search.naver.com/search.naver?where=news&query=%EC%BD%94%EB%A1%9C%EB%82%98&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1025&nso='

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

topic_list = []
text_list = []

# 중앙일보 뉴스 수집
while True:
    
    # 페이지 넘기기
    for j in range(0, 100):
        
        # 페이지 내 크롤링
        for i in range(1, 11):
            
            driver.switch_to.window(driver.window_handles[0])

            driver.implicitly_wait(2)
            time.sleep(1)
            
            # 뉴스 기사 클릭
            try:

                name = driver.find_element_by_xpath('//*[@id="sp_nws' + str(j*10+i) + '"]/div/div/a')
                name.click()
            
            # 페이지 예외처리
            except:
                pass

            driver.switch_to.window(driver.window_handles[-1])


            driver.implicitly_wait(2)
            time.sleep(1)
            
            # 제목, 내용 수집
            try:

                topic = '//*[@id="article_title"]'
                get_topic = driver.find_element_by_xpath(topic)
                topic_list.append(get_topic.text)

                text = '//*[@id="article_body"]'
                get_text = driver.find_element_by_xpath(text)
                text_list.append(get_text.text)
            
            except:
                
                pass

            driver.close()

        driver.switch_to.window(driver.window_handles[0])    
        
        # 다음 페이지로
        next_button = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/a[2]')
        next_button.click()
        
# 데이터 프레임으로 데이터 생성        
news = pd.DataFrame({'제목': topic_list, '내용': text_list})

# csv파일로 쓰기
news.to_csv("data_746.csv")

# 2. 제목 리스트 출력

In [2]:
def enter_document():
    
    import numpy as np
    import pandas as pd
    # 데이터 불러오기
    df = pd.read_csv("data_746.csv")

    # 데이터 cleaning
    import re
    df["내용"] = df["내용"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
    
    # 제목 출력
    print(df['제목'])
    
    # 제목 입력
    title = input('data input : ' )
    
    # 제목에 해당하는 문서 출력
    temp = df[df['제목'] == title]
    print("제목 : ", temp['제목'])
    print("내용 : ", temp['내용'])
    
    return temp
    

In [3]:
enter_document()

0                 전세 줄고 월세 비싸지고…새 임대차법, 세입자 부담 커졌다
1                        임대차법 9개월, 서울 전세 줄고 월세 늘었다
2                朴캠프 또 사퇴…'피해호소인' 3인 이어 '월세인상' 박주민
3                  박주민, 욕먹고 월세 내렸는데…송영길 "역시 박주민답다"
4               "우린 월세전전, LH는 투기전전" 분노한 청년들 촛불 들었다
                          ...                     
741                핑크퐁룸도 생겼어요…'키캉스' 뛰어든 코로나 시대 호텔들
742    당국 "지난해 하루 사망자 847명, 접종 뒤 사망 보도 접종 의사 떨어트려"
743                           [시조가 있는 아침] (70) 어머니
744             인천 남동구, 부모가 안심하는 투명한 어린이집 공개체계 만든다
745                         [시론] 아동학대 초래하는 긴급돌봄 실패
Name: 제목, Length: 746, dtype: object
data input : 임대차법 9개월, 서울 전세 줄고 월세 늘었다
제목 :  1    임대차법 9개월, 서울 전세 줄고 월세 늘었다
Name: 제목, dtype: object
내용 :  1    지난해 7월 말 임대차 2법 전 월세 상한제와 계약갱신청구권 을 시행한 뒤 서울에서...
Name: 내용, dtype: object


# 3. 유사도 높은 문서 제목 출력

In [ ]:
def get_recommendations():
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    
    title = input('문서 제목을 입력하세요 : ')
    
    
    
    # overview에 대해서 tf-idf 수행
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['내용'])
    print(tfidf_matrix.shape)
    
    # 선택한 뉴스의 타이틀로부터 해당되는 인덱스를 받아옵니다. 선택한 뉴스를 가지고 연산.
    indices = pd.Series(df.index, index=df['제목']).drop_duplicates()
    idx = indices[title]
    
    # 모든 뉴스에 대해서 해당 뉴스와의 유사도 구하기
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 뉴스들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 뉴스를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 뉴스의 인덱스를 받아옵니다.
    news_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 뉴스의 제목을 리턴합니다.
    return df['제목'].iloc[news_indices]



In [ ]:
get_recommendations()

# 4. 단어 빈도수

In [124]:
def wordfrequency_top():
    
    from konlpy.tag import Okt
    import re 
    from collections import Counter
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    
    okt=Okt()
    
    # 단어 입력
    word = input('단어를 입력하세요 : ' )
    
    # 제일 빈도가 높은 단어, 개수 column에 추가
    most_1_word_list = []
    most_1_count_list = []
    
    for i in range(len(df)):
        corpus = df['내용'][i]
        noun = okt.nouns(corpus)
        count = Counter(noun)
        most_1_word = count.most_common(1)[0][0]
        most_1_count = count.most_common(1)[0][1]
        most_1_word_list.append(most_1_word)
        most_1_count_list.append(most_1_count)
        
    df['top1'] = most_1_word_list
    df['top1_count'] = most_1_count_list
    
    # 검색한 단어에 맞는 dataframe 생성
    search_list = df[df['top1'] == word]
    
    # 빈도수가 높은 순으로 정렬
    count_sort = search_list.sort_values(by=['top1_count'], axis=0, ascending=False)
    
    # 검색한 단어가 가장 많이 출현한 문서 리스트 출력
    print("문서 출력 : ", count_sort['제목'])
    
    # 제목 입력
    title = input('문서 제목을 입력하세요 : ' )
    
    # overview에 대해서 tf-idf 수행
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['내용'])
    
    # 선택한 뉴스의 타이틀로부터 해당되는 인덱스를 받아옵니다. 선택한 뉴스를 가지고 연산.
    indices = pd.Series(df.index, index=df['제목']).drop_duplicates()
    idx = indices[title]
    
    # 선택한 뉴스의 내용 저장
    corpus = df['내용'][idx]
    
    # 명사 단위 추출 , 형태소 -> okt.morphs()
    noun = okt.nouns(corpus)

    # 단어 빈도 계산
    count = Counter(noun)

    # 빈도 상위 10개 출력
    print("Top ranking word : ", count.most_common(10))

In [125]:
frequency_top_10()

data input : 월세
문서 출력 :  35                  임대차2법 파장, 고액 월세 평균 25만원 뛰었다
36           "괜히 집 샀다 찍힐라" 임대차법에 '부자 월세 세입자' 는다
40                    서울 강남 200만~300만원 월세 확 늘었다
93            [경제통] 금리 내리자 월세 습격, 가난할수록 타격 더 컸다
85           월세 전환 늘자 월세 누르기···“임대인·임차인 싸움 붙이나”
41           도곡렉슬 2억에 월 270만원…월세, 직장인 월급수준으로 뛴다
91              年6000만원 월세받는데 보유세만 1억…다주택자 못버틴다
92              어서와, '월세 파업'은 처음이지?…미국 세입자들 뿔났다
86         전‧월세 전환율 Q&A…“이미 계약한 월세도 초과분 안 내도 돼”
1                     임대차법 9개월, 서울 전세 줄고 월세 늘었다
10      고용·주거 한파에 팍팍한 2030…서울시 “월세 최대 200만원 지원”
56                     [김동호의 시시각각] 월세의 서러움을 아는가
76                   [시론] 주택시장 왜곡하는 전·월세 전환율 족쇄
89                                  [분수대] 월세 시대
83     전·월세 전환율 낮췄지만 “월세로 바꿀 전세 물건 없는데 집주인만 자극”
101           "월세 몸소 실천한다"는 윤준병, 알고보니 지역구 정읍 월세
37          월급 123만원, 7평 원룸 월세 41만원…서울살이 청년들 절규
0              전세 줄고 월세 비싸지고…새 임대차법, 세입자 부담 커졌다
100           윤준병 "월세 사는 사람들 나쁘다고 인식되는 것 옳지 않아"
84              월세 최대 150만원 돌려받도록…與 윤준병, 개정안 발의
79         월세는 

# 5. 단어 사전 (vocab)

In [90]:
# vocab 사전 만들기
vocab = {}

index = 1

for i in range(len(df)):
    corpus = df['내용'][i]
    noun = okt.nouns(corpus)
    for j in range(len(noun)):
        if noun[j] not in vocab.values():
            vocab[index] = noun[j]
            index += 1

In [94]:
# json 파일로 저장
import json

file_path = "./vocab.json"

with open(file_path, 'w') as outfile:
    json.dump(vocab, outfile, indent=4)

In [95]:
# json 파일 읽기
file_path = "./vocab.json"

with open(file_path, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)

{'1': '임대', '2': '차법', '3': '시행', '4': '이후', '5': '서울', '6': '임대차', '7': '시장', '8': '전세', '9': '줄', '10': '월세', '11': '공인', '12': '중개', '13': '사무소', '14': '앞', '15': '시민', '16': '모습', '17': '뉴스', '18': '지난해', '19': '말', '20': '법', '21': '전월세', '22': '제', '23': '계약', '24': '갱신', '25': '청구권', '26': '이', '27': '후', '28': '주택', '29': '것', '30': '또', '31': '개월', '32': '헬리', '33': '시티', '34': '내년', '35': '불안', '36': '상황', '37': '기존', '38': '전셋집', '39': '더', '40': '연장', '41': '수', '42': '행사', '43': '입자', '44': '연쇄', '45': '현상', '46': '우선', '47': '물량', '48': '셋값', '49': '크게', '50': '갑자기', '51': '오른', '52': '감당', '53': '일부', '54': '나', '55': '수요', '56': '수급', '57': '논리', '58': '게다가', '59': '보', '60': '유세', '61': '마련', '62': '위해', '63': '방식', '64': '집주인', '65': '때문', '66': '부동산', '67': '광장', '68': '새', '69': '지난달', '70': '아파트', '71': '전', '72': '거래', '73': '총', '74': '건', '75': '이중', '76': '포함', '77': '집계', '78': '간', '79': '비중', '80': '비', '81': '포인트', '82': '증가', '83': '반면', '84': '로', '85': '

# 최종 프로그램

In [10]:
def get_recommendations():
    import numpy as np
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    
    ### 2번
    # 데이터 불러오기
    df = pd.read_csv("data_746.csv")

    # 데이터 cleaning
    import re
    df["내용"] = df["내용"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
    
    # 제목 출력
    print(df['제목'])
    print("-----------------------")
    
    # 제목 입력
    title = input('data input : ' )
    
    # 제목에 해당하는 문서 출력
    temp = df[df['제목'] == title]
    print("제목 : ", temp['제목'])
    print("내용 : ", temp['내용'])
    print("-----------------------")
    
    ### 3번
    # overview에 대해서 tf-idf 수행
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['내용'])
    
    # 선택한 뉴스의 타이틀로부터 해당되는 인덱스를 받아옵니다. 선택한 뉴스를 가지고 연산.
    indices = pd.Series(df.index, index=df['제목']).drop_duplicates()
    idx = indices[title]
    
    # 모든 뉴스에 대해서 해당 뉴스와의 유사도 구하기
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 뉴스들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 뉴스를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 뉴스의 인덱스를 받아옵니다.
    news_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 뉴스의 제목을 리턴합니다.
    print("유사 문서")
    print(df['제목'].iloc[news_indices])
    print("----------------------------------------")
    
    ## 4번
    from konlpy.tag import Okt
    import re 
    from collections import Counter
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import linear_kernel
    
    okt=Okt()
    
    # 단어 입력
    word = input('단어를 입력하세요 : ' )
    
    # 제일 빈도가 높은 단어, 개수 column에 추가
    most_1_word_list = []
    most_1_count_list = []
    
    for i in range(len(df)):
        corpus = df['내용'][i]
        noun = okt.nouns(corpus)
        count = Counter(noun)
        most_1_word = count.most_common(1)[0][0]
        most_1_count = count.most_common(1)[0][1]
        most_1_word_list.append(most_1_word)
        most_1_count_list.append(most_1_count)
        
    df['top1'] = most_1_word_list
    df['top1_count'] = most_1_count_list
    
    # 검색한 단어에 맞는 dataframe 생성
    search_list = df[df['top1'] == word]
    
    # 빈도수가 높은 순으로 정렬
    count_sort = search_list.sort_values(by=['top1_count'], axis=0, ascending=False)
    
    # 검색한 단어가 가장 많이 출현한 문서 리스트 출력
    print("문서 출력 : ", count_sort['제목'])
    
    # 제목 입력
    title = input('문서 제목을 입력하세요 : ' )
    
    # overview에 대해서 tf-idf 수행
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['내용'])
    
    # 선택한 뉴스의 타이틀로부터 해당되는 인덱스를 받아옵니다. 선택한 뉴스를 가지고 연산.
    indices = pd.Series(df.index, index=df['제목']).drop_duplicates()
    idx = indices[title]
    
    # 모든 뉴스에 대해서 해당 뉴스와의 유사도 구하기
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 뉴스들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 뉴스를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 뉴스의 인덱스를 받아옵니다.
    news_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 뉴스의 제목을 리턴합니다.
    print("유사 문서")
    print(df['제목'].iloc[news_indices])
    print("----------------------------------------")
    
    # 선택한 뉴스의 내용 저장
    corpus = df['내용'][idx]
    
    # 명사 단위 추출 , 형태소 -> okt.morphs()
    noun = okt.nouns(corpus)

    # 단어 빈도 계산
    count = Counter(noun)

    # 빈도 상위 10개 출력
    print("Top ranking word")
    print(count.most_common(10))

In [11]:
get_recommendations()

0                 전세 줄고 월세 비싸지고…새 임대차법, 세입자 부담 커졌다
1                        임대차법 9개월, 서울 전세 줄고 월세 늘었다
2                朴캠프 또 사퇴…'피해호소인' 3인 이어 '월세인상' 박주민
3                  박주민, 욕먹고 월세 내렸는데…송영길 "역시 박주민답다"
4               "우린 월세전전, LH는 투기전전" 분노한 청년들 촛불 들었다
                          ...                     
741                핑크퐁룸도 생겼어요…'키캉스' 뛰어든 코로나 시대 호텔들
742    당국 "지난해 하루 사망자 847명, 접종 뒤 사망 보도 접종 의사 떨어트려"
743                           [시조가 있는 아침] (70) 어머니
744             인천 남동구, 부모가 안심하는 투명한 어린이집 공개체계 만든다
745                         [시론] 아동학대 초래하는 긴급돌봄 실패
Name: 제목, Length: 746, dtype: object
-----------------------
data input : 임대차법 9개월, 서울 전세 줄고 월세 늘었다
제목 :  1    임대차법 9개월, 서울 전세 줄고 월세 늘었다
Name: 제목, dtype: object
내용 :  1    지난해 7월 말 임대차 2법 전 월세 상한제와 계약갱신청구권 을 시행한 뒤 서울에서...
Name: 내용, dtype: object
-----------------------
유사 문서
0             전세 줄고 월세 비싸지고…새 임대차법, 세입자 부담 커졌다
40                   서울 강남 200만~300만원 월세 확 늘었다
41          도곡렉슬 2억에 월 270만원…월세, 직장인 월급수준으로 뛴다
93           [경제통] 금리 